# Module 2 - Part 2

### Working with Structured operations in PySpark

1. Columns and Expressions
1. Filter and Where Conditions
1. Distinct, Drop Duplicated, Order By
1. Rows and Union
1. Adding, Renaming, Dropping Columns
1. Working with missing and "bad" data
1. User-defined functions

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import ArrayType, FloatType, DateType, BooleanType
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

#assign schema
per_schema = StructType([
  StructField("id", IntegerType(), True),
  StructField("first_name", StringType(), True),
  StructField("last_name", StringType(), True),
  StructField("fav_movies", ArrayType(StringType()), True),
  StructField("salary", FloatType(), True),
  StructField("image_url", StringType(), True),
  StructField("date_of_birth", DateType(), True),
  StructField("active", BooleanType(), True)
])

#Load data
json_file_path =  "dbfs:/FileStore/module2/persons.json"
persons_df = (spark.read.json(json_file_path, per_schema, multiLine="True"))


In [0]:
persons_df.printSchema()


root
-- id: integer (nullable = true)
-- first_name: string (nullable = true)
-- last_name: string (nullable = true)
-- fav_movies: array (nullable = true)
 |-- element: string (containsNull = true)
-- salary: float (nullable = true)
-- image_url: string (nullable = true)
-- date_of_birth: date (nullable = true)
-- active: boolean (nullable = true)

In [0]:
persons_df.show(7)
persons_df.show(5, truncate=False)

+---+----------+---------+--------------------+-------+--------------------+-------------+------+
 id|first_name|last_name| fav_movies| salary| image_url|date_of_birth|active|
+---+----------+---------+--------------------+-------+--------------------+-------------+------+
 1| Drucy| Poppy| [I giorni contati]|1463.36|http://dummyimage...| 1991-02-16| true|
 2| Emelyne| Blaza|[Musketeer, The, ...|3006.04|http://dummyimage...| 1991-11-02| false|
 3| Max| Rettie|[The Forgotten Sp...|1422.88|http://dummyimage...| 1990-03-03| false|
 4| Ilario| Kean|[Up Close and Per...|3561.36|http://dummyimage...| 1987-06-09| true|
 5| Toddy| Drexel|[Walk in the Clou...|4934.87|http://dummyimage...| 1992-10-28| true|
 6| Oswald| Petrolli|[Wing and the Thi...|1153.23|http://dummyimage...| 1986-09-02| false|
 7| Adrian| Clarey|[Walking Tall, Pa...|1044.73|http://dummyimage...| 1971-08-24| false|
+---+----------+---------+--------------------+-------+--------------------+-------------+------+
only showing top 7 rows

+---+----------+---------+-------------------------------------+-------+-----------------------------------------------+-------------+------+
id |first_name|last_name|fav_movies |salary |image_url |date_of_birth|active|
+---+----------+---------+-------------------------------------+-------+-----------------------------------------------+-------------+------+
1 |Drucy |Poppy |[I giorni contati] |1463.36|http://dummyimage.com/126x166.png/cc0000/ffffff|1991-02-16 |true |
2 |Emelyne |Blaza |[Musketeer, The, Topralli] |3006.04|http://dummyimage.com/158x106.bmp/cc0000/ffffff|1991-11-02 |false |
3 |Max |Rettie |[The Forgotten Space, Make It Happen]|1422.88|http://dummyimage.com/237x140.jpg/ff4444/ffffff|1990-03-03 |false |
4 |Ilario |Kean |[Up Close and Personal] |3561.36|http://dummyimage.com/207x121.jpg/cc0000/ffffff|1987-06-09 |true |
5 |Toddy |Drexel |[Walk in the Clouds, A] |4934.87|http://dummyimage.com/116x202.png/cc0000/ffffff|1992-10-28 |true |
+---+----------+---------+-------------------------------------+-------+-----------------------------------------------+-------------+------+
only showing top 5 rows

##### 1. Columns and Expressions

In [0]:
from pyspark.sql.functions import col, expr

In [0]:
persons_df.select(col("first_name"), col("last_name"), col("date_of_birth")).show(5)

+----------+---------+-------------+
first_name|last_name|date_of_birth|
+----------+---------+-------------+
 Drucy| Poppy| 1991-02-16|
 Emelyne| Blaza| 1991-11-02|
 Max| Rettie| 1990-03-03|
 Ilario| Kean| 1987-06-09|
 Toddy| Drexel| 1992-10-28|
+----------+---------+-------------+
only showing top 5 rows

In [0]:
persons_df.select(expr("first_name"), expr("last_name"), expr("date_of_birth")).show(5)

+----------+---------+-------------+
first_name|last_name|date_of_birth|
+----------+---------+-------------+
 Drucy| Poppy| 1991-02-16|
 Emelyne| Blaza| 1991-11-02|
 Max| Rettie| 1990-03-03|
 Ilario| Kean| 1987-06-09|
 Toddy| Drexel| 1992-10-28|
+----------+---------+-------------+
only showing top 5 rows

In [0]:
from pyspark.sql.functions import concat_ws

In [0]:
# Concatenate and calculate salary - add 10%
(persons_df.select(concat_ws(' ', col("first_name"), col("last_name")).alias("full_name"),
                 col("salary"),
                 col("salary")* 0.10 + col("salary")).alias("salary_increase")).show(10)

+----------------+-------+-------------------------+
 full_name| salary|((salary * 0.1) + salary)|
+----------------+-------+-------------------------+
 Drucy Poppy|1463.36| 1609.6959838867188|
 Emelyne Blaza|3006.04| 3306.64404296875|
 Max Rettie|1422.88| 1565.1680053710938|
 Ilario Kean|3561.36| 3917.4961181640624|
 Toddy Drexel|4934.87| 5428.35712890625|
 Oswald Petrolli|1153.23| 1268.552978515625|
 Adrian Clarey|1044.73| 1149.202978515625|
Dominica Goodnow|1147.76| 1262.5360107421875|
 Emory Slocomb|1082.11| 1190.3209838867188|
 Jeremias Bode|3472.63| 3819.89287109375|
+----------------+-------+-------------------------+
only showing top 10 rows

In [0]:
(persons_df.select(concat_ws(' ', col("first_name"), col("last_name")).alias("full_name"),
                 col("salary"),
                 expr("salary * 0.10 + salary").alias("salary_increase"))).show(10)

+----------------+-------+------------------+
 full_name| salary| salary_increase|
+----------------+-------+------------------+
 Drucy Poppy|1463.36|1609.6959838867188|
 Emelyne Blaza|3006.04| 3306.64404296875|
 Max Rettie|1422.88|1565.1680053710938|
 Ilario Kean|3561.36|3917.4961181640624|
 Toddy Drexel|4934.87| 5428.35712890625|
 Oswald Petrolli|1153.23| 1268.552978515625|
 Adrian Clarey|1044.73| 1149.202978515625|
Dominica Goodnow|1147.76|1262.5360107421875|
 Emory Slocomb|1082.11|1190.3209838867188|
 Jeremias Bode|3472.63| 3819.89287109375|
+----------------+-------+------------------+
only showing top 10 rows

##### 2. Filter and Where Conditions

In [0]:
#built-in functions: select, order by, group by, where, filter

persons_df.filter("salary <= 3000").show(10)

+---+----------+-----------+--------------------+-------+--------------------+-------------+------+
 id|first_name| last_name| fav_movies| salary| image_url|date_of_birth|active|
+---+----------+-----------+--------------------+-------+--------------------+-------------+------+
 1| Drucy| Poppy| [I giorni contati]|1463.36|http://dummyimage...| 1991-02-16| true|
 3| Max| Rettie|[The Forgotten Sp...|1422.88|http://dummyimage...| 1990-03-03| false|
 6| Oswald| Petrolli|[Wing and the Thi...|1153.23|http://dummyimage...| 1986-09-02| false|
 7| Adrian| Clarey|[Walking Tall, Pa...|1044.73|http://dummyimage...| 1971-08-24| false|
 8| Dominica| Goodnow| [Hearts Divided]|1147.76|http://dummyimage...| 1973-08-27| false|
 9| Emory| Slocomb|[Snake and Crane ...|1082.11|http://dummyimage...| 1974-06-08| true|
 11| Timothy| Ervine|[Land of the Lost...|1147.61|http://dummyimage...| 1971-06-02| false|
 12| Leanora| Gooder|[It's All About L...|1327.02|http://dummyimage...| 1981-12-17| false|
 13| Claiborn| Denham|[McCullin, Max Pa...|2623.33|http://dummyimage...| 1996-03-07| false|
 15| Feodor|Nancekivell| [Monsoon Wedding]|2218.46|http://dummyimage...| 2000-10-07| false|
+---+----------+-----------+--------------------+-------+--------------------+-------------+------+
only showing top 10 rows

In [0]:
persons_df.where("salary <= 3000").show(10)

+---+----------+-----------+--------------------+-------+--------------------+-------------+------+
 id|first_name| last_name| fav_movies| salary| image_url|date_of_birth|active|
+---+----------+-----------+--------------------+-------+--------------------+-------------+------+
 1| Drucy| Poppy| [I giorni contati]|1463.36|http://dummyimage...| 1991-02-16| true|
 3| Max| Rettie|[The Forgotten Sp...|1422.88|http://dummyimage...| 1990-03-03| false|
 6| Oswald| Petrolli|[Wing and the Thi...|1153.23|http://dummyimage...| 1986-09-02| false|
 7| Adrian| Clarey|[Walking Tall, Pa...|1044.73|http://dummyimage...| 1971-08-24| false|
 8| Dominica| Goodnow| [Hearts Divided]|1147.76|http://dummyimage...| 1973-08-27| false|
 9| Emory| Slocomb|[Snake and Crane ...|1082.11|http://dummyimage...| 1974-06-08| true|
 11| Timothy| Ervine|[Land of the Lost...|1147.61|http://dummyimage...| 1971-06-02| false|
 12| Leanora| Gooder|[It's All About L...|1327.02|http://dummyimage...| 1981-12-17| false|
 13| Claiborn| Denham|[McCullin, Max Pa...|2623.33|http://dummyimage...| 1996-03-07| false|
 15| Feodor|Nancekivell| [Monsoon Wedding]|2218.46|http://dummyimage...| 2000-10-07| false|
+---+----------+-----------+--------------------+-------+--------------------+-------------+------+
only showing top 10 rows

In [0]:
persons_df.where((col("salary") <= 3000) & (col("active") == True)).show(10)

+---+----------+---------+--------------------+-------+--------------------+-------------+------+
 id|first_name|last_name| fav_movies| salary| image_url|date_of_birth|active|
+---+----------+---------+--------------------+-------+--------------------+-------------+------+
 1| Drucy| Poppy| [I giorni contati]|1463.36|http://dummyimage...| 1991-02-16| true|
 9| Emory| Slocomb|[Snake and Crane ...|1082.11|http://dummyimage...| 1974-06-08| true|
 16| Margaux| Archbold|[And Now a Word f...|1013.75|http://dummyimage...| 1988-07-29| true|
 26| Clive| Lax| [Rabid]|2126.87|http://dummyimage...| 1981-10-26| true|
 33| Sherline| Primett| [Jungle Fighters]|2309.39|http://dummyimage...| 1972-07-23| true|
 34| Davis| Pinks| [Hounddog]|1337.14|http://dummyimage...| 1989-07-27| true|
 37| Carlen| Sharply|[Dr. Jekyll and M...|2051.85|http://dummyimage...| 2002-06-01| true|
 40| Jordan| Lorant|[Shockproof, Bach...|2183.91|http://dummyimage...| 1979-07-29| true|
 49| Kendricks| Kee| [Flower & Garnet]|2304.39|http://dummyimage...| 1999-11-14| true|
 57| Krystle| Shovell|[Doomsday, Flight...|2260.76|http://dummyimage...| 1987-09-01| true|
+---+----------+---------+--------------------+-------+--------------------+-------------+------+
only showing top 10 rows

In [0]:
from pyspark.sql.functions import year

In [0]:
persons_df.filter((year("date_of_birth") == 2000) | (year("date_of_birth") == 1989)).show()

+---+----------+-----------+--------------------+-------+--------------------+-------------+------+
 id|first_name| last_name| fav_movies| salary| image_url|date_of_birth|active|
+---+----------+-----------+--------------------+-------+--------------------+-------------+------+
 14| Ambrosi| Vidineev|[Wall Street: Mon...|4550.88|http://dummyimage...| 1989-07-20| true|
 15| Feodor|Nancekivell| [Monsoon Wedding]|2218.46|http://dummyimage...| 2000-10-07| false|
 18| Alfie| Hatliffe| [Lord of Tears]| 3893.1|http://dummyimage...| 1989-06-21| true|
 25| Kelcy| Wogdon| [Iron Mask, The]|4512.51|http://dummyimage...| 2000-10-20| true|
 32| Redd| Akenhead|[Century of the D...| 2470.9|http://dummyimage...| 2000-06-05| false|
 34| Davis| Pinks| [Hounddog]|1337.14|http://dummyimage...| 1989-07-27| true|
 61| Shanna| Samples|[Thomas in Love (...| 2703.0|http://dummyimage...| 1989-07-07| false|
 69| Annabell| Doughty|[Entertaining Ang...|2022.57|http://dummyimage...| 2000-09-03| true|
 74| Micky| Umfrey|[Haunted House, T...|1271.82|http://dummyimage...| 1989-07-04| false|
 88| Jobie| Maughan|[Devils on the Do...| 3899.2|http://dummyimage...| 2000-02-07| false|
+---+----------+-----------+--------------------+-------+--------------------+-------------+------+

In [0]:
#nested arrays
from pyspark.sql.functions import array_contains

In [0]:
persons_df.where(array_contains(persons_df.fav_movies, "Land of the Lost")).show()

+---+----------+---------+--------------------+-------+--------------------+-------------+------+
 id|first_name|last_name| fav_movies| salary| image_url|date_of_birth|active|
+---+----------+---------+--------------------+-------+--------------------+-------------+------+
 11| Timothy| Ervine|[Land of the Lost...|1147.61|http://dummyimage...| 1971-06-02| false|
+---+----------+---------+--------------------+-------+--------------------+-------------+------+

##### 3. Distinct, Drop Duplicates, Order By

In [0]:
from pyspark.sql.functions import count, desc

In [0]:
persons_df.select("active").show(10)

+------+
active|
+------+
 true|
 false|
 false|
 true|
 true|
 false|
 false|
 false|
 true|
 true|
+------+
only showing top 10 rows

In [0]:
# get unique values
persons_df.select("active").distinct().show()

+------+
active|
+------+
 true|
 false|
+------+

In [0]:
# let us use orderBy, alias and get Years out 
(persons_df.select(col("first_name"),
                  year(col("date_of_birth")).alias("year"),
                  col("active")).orderBy("year", "first_name")).show(10)

+----------+----+------+
first_name|year|active|
+----------+----+------+
 Adrian|1971| false|
 Feodora|1971| true|
 Sky|1971| false|
 Timothy|1971| false|
 Lucita|1972| true|
 Rodi|1972| false|
 Sherline|1972| true|
 Toddy|1972| true|
 Dominica|1973| false|
 Kelila|1973| true|
+----------+----+------+
only showing top 10 rows

In [0]:
#let  us remove the "duplicates"
dropped_df = (persons_df.select(col("first_name"),
                  year(col("date_of_birth")).alias("year"),
                  col("active")).dropDuplicates(["year","active"]).orderBy("year", "first_name"))

In [0]:
dropped_df.show()

+----------+----+------+
first_name|year|active|
+----------+----+------+
 Adrian|1971| false|
 Feodora|1971| true|
 Rodi|1972| false|
 Sherline|1972| true|
 Dominica|1973| false|
 Kelila|1973| true|
 Balduin|1974| false|
 Emory|1974| true|
 Janean|1975| true|
 Bev|1976| true|
 Franciska|1976| false|
 Johny|1977| false|
 Daveta|1978| false|
 Guthrie|1978| true|
 Maxi|1979| false|
 Melinda|1979| true|
 Carter|1980| false|
 Loralyn|1980| true|
 Clive|1981| true|
 Leanora|1981| false|
+----------+----+------+
only showing top 20 rows

In [0]:
# Order by Year by ASC order
# let us use orderBy, alias and get Years out 
(persons_df.select(col("first_name"),
                  year(col("date_of_birth")).alias("year"),
                  col("active")).orderBy("year", ascending = False)).show(10)

+----------+----+------+
first_name|year|active|
+----------+----+------+
 Daron|2002| true|
 Virgie|2002| true|
 Carlen|2002| true|
 Lorilee|2002| false|
 Maxine|2001| false|
 Feodor|2000| false|
 Kelcy|2000| true|
 Annabell|2000| true|
 Redd|2000| false|
 Jobie|2000| false|
+----------+----+------+
only showing top 10 rows

##### 4. Rows and union

In [0]:
from pyspark.sql import Row

In [0]:
person_row = Row(101, "Robert","Golob",["Lord of The Rings 1", "Lord of The Rings 2"], 10000.51, "http://gibanje-svoboda.si","1967-02-22", True)

In [0]:
# Create a variable that will hold list of rows
#person_rows_list = []
person_rows_list = [Row(102, "Mike","McCready",["Lord of The Rings 1", "Lord of The Rings 2"], 10000.51, "http://PearlJam.com","1967-12-22", True),
                    Row(103, "Eddie","Vedder",["Lord of The Rings 1", "Lord of The Rings 2"], 10000.51, "http://PearLJam.com","1967-06-22", True)]

In [0]:
person_rows_list.append(person_row)

In [0]:
print(person_rows_list)

[<Row(102, 'Mike', 'McCready', ['Lord of The Rings 1', 'Lord of The Rings 2'], 10000.51, 'http://PearlJam.com', '1967-012-22', True)>, <Row(103, 'Eddie', 'Vedder', ['Lord of The Rings 1', 'Lord of The Rings 2'], 10000.51, 'http://PearLJam.com', '1967-06-22', True)>, <Row(101, 'Robert', 'Golob', ['Lord of The Rings 1', 'Lord of The Rings 2'], 10000.51, 'http://gibanje-svoboda.si', '1967-02-22', True)>]

In [0]:
person_row[2]

Out[36]: 'Golob'

In [0]:
# infer schema
new_persons_df = spark.createDataFrame(person_rows_list, ["id","first_name", "last_name", "fav_movies", "salary", "image_url", "date_of_birth","active"])

In [0]:
#union all data / combine it with existing dataframe
add_persons_df = persons_df.union(new_persons_df)
add_persons_df.sort(desc("id")).show(10)

+---+----------+---------+--------------------+------------------+--------------------+-------------+------+
 id|first_name|last_name| fav_movies| salary| image_url|date_of_birth|active|
+---+----------+---------+--------------------+------------------+--------------------+-------------+------+
103| Eddie| Vedder|[Lord of The Ring...| 10000.51| http://PearLJam.com| 1967-06-22| true|
102| Mike| McCready|[Lord of The Ring...| 10000.51| http://PearlJam.com| 1967-012-22| true|
101| Robert| Golob|[Lord of The Ring...| 10000.51|http://gibanje-sv...| 1967-02-22| true|
100| Virgie| Domanski|[Horseman, The, S...| 2165.929931640625|http://dummyimage...| 2002-01-05| true|
 99| Rozalie| Wannop|[Suddenly, The No...|1259.6400146484375|http://dummyimage...| 1997-03-25| false|
 98| Davin| Labb|[Viva Riva!, Kill...| 1452.739990234375|http://dummyimage...| 1988-01-27| true|
 97| Rodi| Farnan|[Code, The (Menta...| 2325.8798828125|http://dummyimage...| 1972-01-04| false|
 96| Dew| Coopland| [Rush]| 2725.56005859375|http://dummyimage...| 1986-11-14| false|
 95| Cobb| MacLure|[Storage 24, His ...|1621.1700439453125|http://dummyimage...| 1994-06-28| false|
 94| Bennie| Knight|[House on Carroll...| 2370.239990234375|http://dummyimage...| 1977-08-27| false|
+---+----------+---------+--------------------+------------------+--------------------+-------------+------+
only showing top 10 rows

#### 5. Adding, Renaming, Dropping Columns

In [0]:
from pyspark.sql.functions import round

In [0]:
#add new column "Salary increase"
aug_persons_df1 = persons_df.withColumn("salary_increase", expr("salary * 0.10 + salary"))
aug_persons_df1.show(10)

+---+----------+---------+--------------------+-------+--------------------+-------------+------+------------------+
 id|first_name|last_name| fav_movies| salary| image_url|date_of_birth|active| salary_increase|
+---+----------+---------+--------------------+-------+--------------------+-------------+------+------------------+
 1| Drucy| Poppy| [I giorni contati]|1463.36|http://dummyimage...| 1991-02-16| true|1609.6959838867188|
 2| Emelyne| Blaza|[Musketeer, The, ...|3006.04|http://dummyimage...| 1991-11-02| false| 3306.64404296875|
 3| Max| Rettie|[The Forgotten Sp...|1422.88|http://dummyimage...| 1990-03-03| false|1565.1680053710938|
 4| Ilario| Kean|[Up Close and Per...|3561.36|http://dummyimage...| 1987-06-09| true|3917.4961181640624|
 5| Toddy| Drexel|[Walk in the Clou...|4934.87|http://dummyimage...| 1992-10-28| true| 5428.35712890625|
 6| Oswald| Petrolli|[Wing and the Thi...|1153.23|http://dummyimage...| 1986-09-02| false| 1268.552978515625|
 7| Adrian| Clarey|[Walking Tall, Pa...|1044.73|http://dummyimage...| 1971-08-24| false| 1149.202978515625|
 8| Dominica| Goodnow| [Hearts Divided]|1147.76|http://dummyimage...| 1973-08-27| false|1262.5360107421875|
 9| Emory| Slocomb|[Snake and Crane ...|1082.11|http://dummyimage...| 1974-06-08| true|1190.3209838867188|
 10| Jeremias| Bode|[Farewell to Arms...|3472.63|http://dummyimage...| 1997-08-02| true| 3819.89287109375|
+---+----------+---------+--------------------+-------+--------------------+-------------+------+------------------+
only showing top 10 rows

In [0]:
#check if this column is "really" in DataFrame
aug_persons_df1.columns

Out[43]: ['id',
 'first_name',
 'last_name',
 'fav_movies',
 'salary',
 'image_url',
 'date_of_birth',
 'active',
 'salary_increase']

In [0]:
# create 1 new column for year of birth and rename fav_movies, round the values and drop one column
aug_persons_df2 = (aug_persons_df1
                  .withColumn("birth_year", year("date_of_birth"))
                  .withColumnRenamed("fav_movies", "movies")
                  .withColumn("salary_x10", round(col("salary_increase"),2))
                  .drop("salary_increase"))

In [0]:
aug_persons_df2.show(10)

+---+----------+---------+--------------------+-------+--------------------+-------------+------+----------+----------+
 id|first_name|last_name| movies| salary| image_url|date_of_birth|active|birth_year|salary_x10|
+---+----------+---------+--------------------+-------+--------------------+-------------+------+----------+----------+
 1| Drucy| Poppy| [I giorni contati]|1463.36|http://dummyimage...| 1991-02-16| true| 1991| 1609.7|
 2| Emelyne| Blaza|[Musketeer, The, ...|3006.04|http://dummyimage...| 1991-11-02| false| 1991| 3306.64|
 3| Max| Rettie|[The Forgotten Sp...|1422.88|http://dummyimage...| 1990-03-03| false| 1990| 1565.17|
 4| Ilario| Kean|[Up Close and Per...|3561.36|http://dummyimage...| 1987-06-09| true| 1987| 3917.5|
 5| Toddy| Drexel|[Walk in the Clou...|4934.87|http://dummyimage...| 1992-10-28| true| 1992| 5428.36|
 6| Oswald| Petrolli|[Wing and the Thi...|1153.23|http://dummyimage...| 1986-09-02| false| 1986| 1268.55|
 7| Adrian| Clarey|[Walking Tall, Pa...|1044.73|http://dummyimage...| 1971-08-24| false| 1971| 1149.2|
 8| Dominica| Goodnow| [Hearts Divided]|1147.76|http://dummyimage...| 1973-08-27| false| 1973| 1262.54|
 9| Emory| Slocomb|[Snake and Crane ...|1082.11|http://dummyimage...| 1974-06-08| true| 1974| 1190.32|
 10| Jeremias| Bode|[Farewell to Arms...|3472.63|http://dummyimage...| 1997-08-02| true| 1997| 3819.89|
+---+----------+---------+--------------------+-------+--------------------+-------------+------+----------+----------+
only showing top 10 rows

#### 6. Cleaning and working with missing \ bad data

In [0]:
bad_movies_list = [Row(None, None, None),
                   Row(None, None, 2022),
                   Row("John McTyre", "Another Great Movie", None),
                   Row(None, "Amazing Movie", 2021),
                   Row("May Field", None, 2011),
                   Row("Vlad Stopin", "We don't want war", 2022)
                  ]

In [0]:
bad_movies_list

Out[48]: [<Row(None, None, None)>,
 <Row(None, None, 2022)>,
 <Row('John McTyre', 'Another Great Movie', None)>,
 <Row(None, 'Amazing Movie', 2021)>,
 <Row('May Field', None, 2011)>,
 <Row('Vlad Stopin', "We don't want war", 2022)>]

In [0]:
bad_movies_columns = ["actor_name", "movie_title", "produced_year"]

In [0]:
bad_movies_df = spark.createDataFrame(bad_movies_list, schema=bad_movies_columns)

In [0]:

bad_movies_df.show()

+-----------+-------------------+-------------+
 actor_name| movie_title|produced_year|
+-----------+-------------------+-------------+
 null| null| null|
 null| null| 2022|
John McTyre|Another Great Movie| null|
 null| Amazing Movie| 2021|
 May Field| null| 2011|
Vlad Stopin| We don't want war| 2022|
+-----------+-------------------+-------------+

In [0]:
#we see null values or rows with null values 
bad_movies_df.na.drop().show()

+-----------+-----------------+-------------+
 actor_name| movie_title|produced_year|
+-----------+-----------------+-------------+
Vlad Stopin|We don't want war| 2022|
+-----------+-----------------+-------------+

In [0]:
bad_movies_df.na.drop("any").show()

+-----------+-----------------+-------------+
 actor_name| movie_title|produced_year|
+-----------+-----------------+-------------+
Vlad Stopin|We don't want war| 2022|
+-----------+-----------------+-------------+

In [0]:
#drop just rows that are all empty
bad_movies_df.na.drop("all").show()

+-----------+-------------------+-------------+
 actor_name| movie_title|produced_year|
+-----------+-------------------+-------------+
 null| null| 2022|
John McTyre|Another Great Movie| null|
 null| Amazing Movie| 2021|
 May Field| null| 2011|
Vlad Stopin| We don't want war| 2022|
+-----------+-------------------+-------------+

In [0]:
#get DataFrame where actor_name is not null
bad_movies_df.filter(col("actor_name").isNull() != True).show()

+-----------+-------------------+-------------+
 actor_name| movie_title|produced_year|
+-----------+-------------------+-------------+
John McTyre|Another Great Movie| null|
 May Field| null| 2011|
Vlad Stopin| We don't want war| 2022|
+-----------+-------------------+-------------+

In [0]:
#get DataFrame where actor_name is null
bad_movies_df.filter(col("actor_name").isNull() != False).show()

+----------+-------------+-------------+
actor_name| movie_title|produced_year|
+----------+-------------+-------------+
 null| null| null|
 null| null| 2022|
 null|Amazing Movie| 2021|
+----------+-------------+-------------+

In [0]:
#describe the values
bad_movies_df.describe("produced_year").show()

+-------+-----------------+
summary| produced_year|
+-------+-----------------+
 count| 4|
 mean| 2019.0|
 stddev|5.354126134736337|
 min| 2011|
 max| 2022|
+-------+-----------------+

#### 7. Create user-defined function 
Using spark sql function because, you can not use Python functions

In [0]:
from pyspark.sql.functions import udf

In [0]:
students_list = [("John", 85),
                ("Mark", 90),
                ("Mike", 109),
                ("Marry", 79)
                ]

In [0]:
students_columns = ["name", "score"]

In [0]:
students_df = spark.createDataFrame(students_list, schema = students_columns)

In [0]:
students_df.show()

+-----+-----+
 name|score|
+-----+-----+
 John| 85|
 Mark| 90|
 Mike| 109|
Marry| 79|
+-----+-----+

In [0]:
# create python function
def LetterGrade(score:int):
  grade = ''
  if score > 100:
    grade = "Uau"
  elif score >= 90:
    grade = "A"
  elif score >= 80:
    grade = "B"
  elif score >= 70:
    grade = "C"
  else:
    grade = "F"
  return grade

In [0]:
#before converting to spark function, lets test the function
print(LetterGrade(86))

B

In [0]:
letterGradeUDF = udf(LetterGrade)

In [0]:
students_df.select("name", "score", letterGradeUDF("score").alias("GradeLetter")).show(10)

+-----+-----+-----------+
 name|score|GradeLetter|
+-----+-----+-----------+
 John| 85| B|
 Mark| 90| A|
 Mike| 109| Uau|
Marry| 79| C|
+-----+-----+-----------+

In [0]:
#creating Spark UDF for converting string to unix-timestamp
from pyspark.sql import Row
from pyspark.sql.functions import unix_timestamp, to_timestamp


from pyspark.sql import SparkSession
spark = (SparkSession.builder.appName("TimeToUnix").getOrCreate())

spark.parallelize([Rows(dt='2022_03_08 11_44_12')]).toDF().withColumn("parsed", to_timestamp("dt", "yyyy_MM_dd HH_mm_ss")).show(1, False)